In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from salishsea_tools import evaltools as et
import datetime as dt
import matplotlib as mpl
import netCDF4 as nc

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

%matplotlib inline

In [2]:
data = pd.read_csv('/ocean/atall/MOAD/Obs/SalishCruise_dataProduct_2015to2018_fromKaryn.csv')
PATH= '/results2/SalishSea/nowcast-green.202111/'
data['dtUTC']=pd.to_datetime(data['dtUTC']) 

In [3]:
def matchData(
    data,
    filemap,
    fdict,
    mod_start=None,
    mod_end=None,
    mod_nam_fmt='nowcast',
    mod_basedir='/results2/SalishSea/nowcast-green.202111/',
    mod_flen=1,
    method='bin',
    meshPath=None,
    maskname='tmask',
    wrapSearch=False,
    fastSearch=False,
    wrapTol=1,
    e3tvar='e3t',
    fid=None,
    sdim=3,
    quiet=False,
    preIndexed=False
    ):
    """Given a discrete sample dataset, find match model output

    note: only one grid mask is loaded so all model variables must be on same grid; defaults to tmask;
        call multiple times for different grids (eg U,W)

    :arg data: pandas dataframe containing data to compare to. Must include the following:
        'dtUTC': column with UTC date and time
        'Lat': decimal latitude
        'Lon': decimal longitude
        'Z': depth, positive     NOT  required if method='ferry' or sdim=2
    :type :py:class:`pandas.DataFrame`

    :arg dict filemap: dictionary mapping names of model variables to filetypes containing them

    :arg dict fdict: dictionary mapping filetypes to their time resolution in hours

    :arg mod_start: first date of time range to match
    :type :py:class:`datetime.datetime`

    :arg mod_end: end of date range to match (not included)
    :type :py:class:`datetime.datetime`

    :arg str mod_nam_fmt: naming format for model files. options are 'nowcast' or 'long'
        'nowcast' example: 05may15/SalishSea_1h_20150505_20150505_ptrc_T.nc
        'long' example: SalishSea_1h_20150206_20150804_ptrc_T_20150427-20150506.nc
                    'long' will recursively search subdirectories (to match Vicky's storage style)

    :arg str mod_basedir: path to search for model files; defaults to nowcast-green

    :arg int mod_flen: length of model files in days; defaults to 1, which is how nowcast data is stored

    :arg str method: method to use for matching. options are:

        'bin'- return model value from grid/time interval containing observation
        'vvlBin' - same as 'bin' but consider tidal change in vertical grid
        'vvlZ' - consider tidal change in vertical grid and interpolate in the vertical
        'ferry' - match observations to top model layer
        'vertNet' - match observations to mean over a vertical range defined by
                    Z_upper and Z_lower; first try will include entire cell containing end points
                    and use e3t_0 rather than time-varying e3t

    :arg str meshPath: path to mesh file; defaults to None, in which case set to:
            '/results/forcing/atmospheric/GEM2.5/operational/ops_y2015m01d01.nc' if maskName is 'ops'
            '/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc' else (SalishSeaCast)

    :arg str maskName: variable name for mask in mesh file (check code for consistency if not tmask)
                       for ops vars use 'ops'

    :arg boolean wrapSearch: if True, use wrapper on find_closest_model_point that assumes
                             nearness of subsequent values

    :arg int wrapTol: assumed search radius from previous grid point if wrapSearch=True

    :arg str e3tvar: name of tgrid thicknesses variable; only for method=interpZe3t, which only works on t grid

    :arg Dataset fid: optionally include name of a single dataset when looping is not necessary and all matches come from
        a single file

    :arg int sdim: optionally enter number of spatial dimensions (must be the same for all variables per call);
        defaults to 3; use to match to 2d fields like ssh

    :arg boolean quiet: if True, suppress non-critical warnings

    :arg boolean preIndexed: set True if horizontal  grid indices already in input dataframe; for
               speed; not implemented with all options
    """
    # define dictionaries of mesh lat and lon variables to use with different grids:
    lonvar={'tmask':'nav_lon','umask':'glamu','vmask':'glamv','fmask':'glamf'}
    latvar={'tmask':'nav_lat','umask':'gphiu','vmask':'gphiv','fmask':'gphif'}

    # check that required columns are in dataframe:
    if method == 'ferry' or sdim==2:
        reqsubset=['dtUTC','Lat','Lon']
        if preIndexed:
            reqsubset=['dtUTC','i','j']
    elif method == 'vertNet':
        reqsubset=['dtUTC','Lat','Lon','Z_upper','Z_lower']
        if preIndexed:
            reqsubset=['dtUTC','i','j','Z_upper','Z_lower']
    else:
        reqsubset=['dtUTC','Lat','Lon','Z']
        if preIndexed:
            reqsubset=['dtUTC','i','j','k']
    if not set(reqsubset) <= set(data.keys()):

        raise Exception('{} missing from data'.format([el for el in set(reqsubset)-set(data.keys())],'%s'))

    fkeysVar=list(filemap.keys()) # list of model variables to return
    # don't load more files than necessary:
    ftypes=list(fdict.keys())
    for ikey in ftypes:
        if ikey not in set(filemap.values()):
            fdict.pop(ikey)
    if len(set(filemap.values())-set(fdict.keys()))>0:
        print('Error: file(s) missing from fdict:',set(filemap.values())-set(fdict.keys()))
    ftypes=list(fdict.keys()) # list of filetypes to containing the desired model variables
    # create inverted version of filemap dict mapping file types to the variables they contain
    filemap_r=dict()
    for ift in ftypes:
        filemap_r[ift]=list()
    for ikey in filemap:
        filemap_r[filemap[ikey]].append(ikey)

    # if mod_start and mod_end not provided, use min and max of data datetimes
    if mod_start is None:
        mod_start=np.min(data['dtUTC'])
        print(mod_start)
    if mod_end is None:
        mod_end=np.max(data['dtUTC'])
        print(mod_end)
    # adjustments to data dataframe to avoid unnecessary calculations
    data=data.loc[(data.dtUTC>=mod_start)&(data.dtUTC<mod_end)].copy(deep=True)
    data=data.dropna(how='any',subset=reqsubset) #.dropna(how='all',subset=[*varmap.keys()])
    if maskname=='ops':
        # set default mesh file for ops data (atmos forcing)
        if meshPath==None:
            meshPath='/results/forcing/atmospheric/GEM2.5/operational/ops_y2015m01d01.nc'
        # load lat, lon, and mask (all ones for ops - no land in sky)
        with nc.Dataset(meshPath) as fmesh:
            navlon=np.squeeze(np.copy(fmesh.variables['nav_lon'][:,:]-360))
            navlat=np.squeeze(np.copy(fmesh.variables['nav_lat'][:,:]))
        omask=np.expand_dims(np.ones(np.shape(navlon)),axis=(0,1))
        nemops='GEM2.5'
    else:
        # set default mesh file for SalishSeaCast data
        if meshPath==None:
            meshPath='/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc'
        # load lat lon and ocean mask
        with nc.Dataset(meshPath) as fmesh:
            omask=np.copy(fmesh.variables[maskname])
            navlon=np.squeeze(np.copy(fmesh.variables[lonvar[maskname]][:,:]))
            navlat=np.squeeze(np.copy(fmesh.variables[latvar[maskname]][:,:]))
            if method == 'vertNet':
                e3t0=np.squeeze(np.copy(fmesh.variables['e3t_0'][0,:,:,:]))
                if maskname != 'tmask':
                    print('Warning: Using tmask thickness for variable on different grid')
        nemops='NEMO'

    # handle horizontal gridding as necessary; make sure data is in order of ascending time
    if not preIndexed:
        # find location of each obs on model grid and add to data as additional columns 'i' and 'j'
        data=et._gridHoriz(data,omask,navlon,navlat,wrapSearch,wrapTol,fastSearch, quiet=quiet,nemops=nemops)
        data=data.sort_values(by=[ix for ix in ['dtUTC','Z','j','i'] if ix in reqsubset]) # preserve list order
    else:
        data=data.sort_values(by=[ix for ix in ['dtUTC','k','j','i'] if ix in reqsubset]) # preserve list order
    data.reset_index(drop=True,inplace=True)

    # set up columns to accept model values; prepend 'mod' to distinguish from obs names
    for ivar in filemap.keys():
        data['mod_'+ivar]=np.full(len(data),np.nan)

    # create dictionary of dataframes of filename, start time, and end time for each file type
    flist=dict()
    for ift in ftypes:
        flist[ift]=et.index_model_files(mod_start,mod_end,mod_basedir,mod_nam_fmt,mod_flen,ift,fdict[ift])

    # call a function to carry out vertical matching based on specified method
    if method == 'bin':
        data = _binmatch(data,flist,ftypes,filemap_r,omask,maskP,sdim,preIndexed=preIndexed)
    elif method == 'ferry':
        print('data is matched to shallowest model level')
        data = _ferrymatch(data,flist,ftypes,filemap_r,omask,fdict)
    elif method == 'vvlZ':
        data = _interpvvlZ(data,flist,ftypes,filemap,filemap_r,omask,fdict,e3tvar)
    elif method == 'vvlBin':
        data= _vvlBin(data,flist,ftypes,filemap,filemap_r,omask,fdict,e3tvar)
    elif method == 'vertNet':
        data = _vertNetmatch(data,flist,ftypes,filemap_r,omask,e3t0,maskP)
    elif method == 'salinity':
        print('Matching by salinity...')
        data = _salinityMatch(data,flist,ftypes,filemap_r,omask,fdict)
    else:
        print('option '+method+' not written yet')
        return
    data.reset_index(drop=True,inplace=True)
    return data

In [4]:
def _salinityMatch(data, flist, ftypes, filemap_r, omask, fdict):
    import xarray as xr
    import numpy as np
    from tqdm import tqdm

    #varname = 'vosaline'
    matched_salinities = []

    # Find which filetype contains salinity
    salinity_var, salinity_ftype = None, None
    for ftype in ftypes:
        for var in filemap_r[ftype]:
            if 'sal' in var.lower():  ## was just 'sal' before
                salinity_var = var
                salinity_ftype = ftype
                break
        if salinity_var:
            break
    if not salinity_var:
        raise ValueError("No salinity variable found in filemap_r.")

    salinity_files = flist[salinity_ftype]
    salinity_files.columns = ['fname', 'start', 'end']

    # Cache for xarray datasets
    dataset_cache = {}

    for idx, row in tqdm(data.iterrows(), total=len(data)):
        obs_time = row['dtUTC']
        obs_sal = row['Sal (g kg-1)']
        j, i = int(row['j']), int(row['i'])
        k = None

        # Step 1: Find matching salinity depth
        for _, mf in salinity_files.iterrows():
            if mf['start'] <= obs_time < mf['end']:
                fname = mf['fname']
                if fname not in dataset_cache:
                    dataset_cache[fname] = xr.open_dataset(fname)
                ds = dataset_cache[fname]

                try:
                    # Select time (nearest if needed), then slice j,i
                    sel = ds[salinity_var].sel(time_counter=obs_time, method='nearest')
                    sal_profile = sel[:, j, i].values  # depth profile

                    if np.isnan(sal_profile).all():
                        matched_salinities.append(np.nan)
                        k = None
                    else:
                        sal_diff = np.abs(sal_profile - obs_sal)
                        k = np.nanargmin(sal_diff)
                        matched_salinities.append(sal_profile[k])
                except Exception as e:
                    print(f"Error reading salinity at {fname}: {e}")
                    matched_salinities.append(np.nan)
                    k = None
                break

        if k is None:
            # Fill all variables with NaN
            for ft in ftypes:
                for var in filemap_r[ft]:
                    data.at[idx, f'mod_{var}'] = np.nan
            continue

        # Step 2: Grab each variable at (time, k, j, i) using xarray
        for ft in ftypes:
            var_files = flist[ft]
            var_files.columns = ['fname', 'start', 'end']

            for _, mf in var_files.iterrows():
                if mf['start'] <= obs_time < mf['end']:
                    fname = mf['fname']
                    if fname not in dataset_cache:
                        dataset_cache[fname] = xr.open_dataset(fname)
                    ds = dataset_cache[fname]

                    for var in filemap_r[ft]:
                        try:
                            sel = ds[var].sel(time_counter=obs_time, method='nearest')
                            val = sel[k, j, i].item()
                        except Exception:
                            val = np.nan
                        data.at[idx, f'mod_{var}'] = val
                    break

    # Close all xarray datasets
    for ds in dataset_cache.values():
        ds.close()

    data['matched_salinity'] = matched_salinities
    return data

In [ ]:
matched_data15 = matchData(
    data=data,  # your observations DataFrame
    filemap={'dissolved_inorganic_carbon':'chem_T','total_alkalinity':'chem_T','dissolved_oxygen':'chem_T',
         'votemper':'grid_T','vosaline':'grid_T'
        },  # tell it which model variable to match
    fdict={'chem_T':1,'grid_T':1},  # model file timestep in hours
    mod_start=dt.datetime(2015,1,1),
    mod_end=dt.datetime(2015,12,31),
    method='salinity')
matched_data15.to_csv('/ocean/atall/MOAD/ObsModel/202111/ObsModel_202111_bySalinityKaryn_20150101_20151231.csv') 

Matching by salinity...


100%|██████████| 915/915 [01:08<00:00, 13.37it/s]


In [ ]:
matched_data16 = matchData(
    data=data,  # your observations DataFrame
    filemap={'dissolved_inorganic_carbon':'chem_T','total_alkalinity':'chem_T','dissolved_oxygen':'chem_T',
         'votemper':'grid_T','vosaline':'grid_T'
        },  # tell it which model variable to match
    fdict={'chem_T':1,'grid_T':1},  # model file timestep in hours
    mod_start=dt.datetime(2016,1,1),
    mod_end=dt.datetime(2016,12,31),
    method='salinity')
matched_data16.to_csv('/ocean/atall/MOAD/ObsModel/202111/ObsModel_202111_bySalinityKaryn_20160101_20161231.csv') 

Matching by salinity...


100%|██████████| 765/765 [00:52<00:00, 14.49it/s]


In [ ]:
matched_data17 = matchData(
    data=data,  # your observations DataFrame
    filemap={'dissolved_inorganic_carbon':'chem_T','total_alkalinity':'chem_T','dissolved_oxygen':'chem_T',
         'votemper':'grid_T','vosaline':'grid_T'
        },  # tell it which model variable to match
    fdict={'chem_T':1,'grid_T':1},  # model file timestep in hours
    mod_start=dt.datetime(2017,1,1),
    mod_end=dt.datetime(2017,12,31),
    method='salinity')
matched_data17.to_csv('/ocean/atall/MOAD/ObsModel/202111/ObsModel_202111_bySalinityKaryn_20170101_20171231.csv') 

Matching by salinity...


100%|██████████| 916/916 [00:55<00:00, 16.38it/s]


In [ ]:
matched_data18 = matchData(
    data=data,  # your observations DataFrame
    filemap={'dissolved_inorganic_carbon':'chem_T','total_alkalinity':'chem_T','dissolved_oxygen':'chem_T',
         'votemper':'grid_T','vosaline':'grid_T'
        },  # tell it which model variable to match
    fdict={'chem_T':1,'grid_T':1},  # model file timestep in hours
    mod_start=dt.datetime(2018,1,1),
    mod_end=dt.datetime(2018,12,31),
    method='salinity')
matched_data18.to_csv('/ocean/atall/MOAD/ObsModel/202111/ObsModel_202111_bySalinityKaryn_20180101_20181231.csv') 

Matching by salinity...


100%|██████████| 1035/1035 [01:01<00:00, 16.84it/s]
